# Vehicle Detection & Confirmation System
### Version 2
Summer 2023 

Created by Vikram Anantha \
Continued from Ben Dwyer's code \
July 27 2023

## Summary
(Same as V1) \
This code is meant to be implemented into Road-Side Systems (RSSs), like traffic cameras, such that it can communicate with vehicles, especially Autonomous Vehicles (AVs) to verify their identity.

### Background

(Same as V1) \
One problem that might arise when AVs communicate with RSSs is that a hacker with malicious intent can join the same channel and communicate with the RSSs as if they were the vehicle. To combat this, the RSS can command the vehicle to confirm its identity by performing a specific task. Examples of this include:
 + Displaying a specific pattern on a screen, similar to a QR code
 + Flashing headlights in a specific pattern
 + Making a sound in a specific pattern

Once the vehicle performs the task, the RSS can confirm this has been done visually, thus confirming the identiy of the vehicle. This visual confirmation, a form of Two Factor Authentication (2FA) is the premise of this code.

### Overview of Version 2

Here is an overview of the main simple code:
~~~
while loop forever:
   read in the frame from the ZED camera
   detect the vehicle in the frame
   process the frame:
      save a cropped version of the frame of just the vehicle
      greyscale the cropped image
      convert the image to a binary image (any pixel above a brightness is white, any below is black); I call it "thresholding" the image
      find the Mean Image Brightness (MIB)
      high MIBs means brighter image, so the headlight is on, lower means dimmer means headlights off
   add the on/off state to a list
~~~

### Changes from V1

These were the shortcomings from Version 1:
 1. When using live feed, there is about a second of delay per frame to detect if a vehicle is present, and to recognize there is no bounding box. It is much more time to recognize the array in the bounding box, which needs to be done for each vehicle.
 2. The code runs a for loop to go through each vehicle and detect the pattern, meaning it verifies each vehicle one by one. Because each vehicle takes a long time to be verified, when multiple vehicles are present, the system will take a _very_ long time.
 3. When using a webcam, or a dedicated camera for the computer (as in not a smartphone camera, which uses HDR to make screens appear normal with everything else), the Arduino display is super bright, and cannot be recognized. However, this doesn't really matter as much, since in the real environment, having a screen on the windshield won't be implemented. Instead, it would most likely leverage the headlights, having them flash a pattern across time, rather than display a pattern across space.

Version 2 solves these problems in these ways:
 1. In this Version, the code only detects if there is a vehicle present, so only using 1 model instead of 2. This should cut the latency by about half. In addition, this code has the model on device, meaning it can utilize the extensive hardware to run the model faster. This cuts the latency by half again.
 2. This part isn't really fixed, since it still has to go through every vehicle to detect their pattern verification
 3. This version does not need a camera with HDR, in fact it actually leverages the HDR-less camera. When the headlights are shining, the camera overexposes them, meaning they will be much brighter than everything else. This makes it much easier to threshold the image and calculate the average brightness.

# Code and Reports
In this file, I have the code that I wrote (or borrowed), however I also inclde some mid-way reports, either what I wrote at the end of the day to get my thoughts in order, or what I sent to Dajiang at the end of the day. I kept this in to show my thought process and all the steps taken to make the final algorithm

### Pre Version 2 reports

#### Meeting with Dajiang Suo [Jul 24 2023]




ZED obj detection should be faster
 - https://www.stereolabs.com/docs/object-detection/
 - Detect vehicle + detect headlights + detect headlight pattern
 - ^^ simultaneously
 - Api provided by zed camera to do categorization
 - Look through https://www.stereolabs.com/docs/object-detection/custom-od/
 - To get obj detection and custom detection


Action-State Joint Learning-Based Vehicle Taillight
 - https://arxiv.org/pdf/1906.03683.pdf
 - Recognition in Diverse Actual Traffic Scenes


Document all these steps rigorouly


Another Paper
 - https://github.com/TechToker/CarLightSignalsDetection
 - ^^ sliding window of frames?


#### Todo (Jul 27 2023):


 + implement Zed camera based object detection
 + run a model to use headlights as the pattern recognition
    + first get that to run on a given video
    + then write arduino code to make the display act like headlights
    + then get it to work in real time
    + then see if the pattern recognition is needed with ML or just if statements
 + try to get multiple vehicles to work fast (idk how yet)
    + zed api?
 + Priority 1: documentation

## Imports and setup

In [ ]:
import glob
from roboflow import Roboflow
from ultralytics import YOLO
import time
import cv2
import numpy as np
from datetime import datetime
import os
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis
import random

BENS_RF_APIKEY = "2Vs9PCO5LGDCSkI0huRq"
BENS_RF_WORKSPACE = "meng-thesis-5fidi"
BENS_RF_PROJECT = "rc-vehicle-detection_v2"
ONOFF_THRES = 8
HL_THRESHOLD = 200

avg = lambda x: sum(x)/len(x)
def export_video(input_val, output_val, fps=25):
    # stich images together into video
    os.system(f"ffmpeg -y -framerate {fps} -pattern_type glob -i '{input_val}' -c:v libx264 -r 30 -pix_fmt yuv420p {output_val}")
def save_frames(image, dir):
    current_time = datetime.now().strftime("%H_%M_%S_%f")
    cv2.imwrite(dir % current_time, image)

## Other Codes

### Zed Camera Code: Object Detection Birds Eye

In [ ]:
# This code is borrowed from the ZED Camera API, which has this and many more codes here
# https://github.com/stereolabs/zed-sdk/tree/master/tutorials/tutorial%206%20-%20object%20detection/python


########################################################################
#
# Copyright (c) 2022, STEREOLABS.
#
# All rights reserved.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
# "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
# LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR
# A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT
# OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
# SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT
# LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE,
# DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY
# THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
# (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
########################################################################

"""
    This sample demonstrates how to capture 3D point cloud and detected objects
    with the ZED SDK and display the result in an OpenGL window.
"""

import sys
import numpy as np
import cv2
import pyzed.sl as sl
# import helpercode.ogl_viewer.viewer as gl
import helpercode.cv_viewer.tracking_viewer as cv_viewer
from helpercode.batch_system_handler import *
# because these files are in some random folder, I copied the important ones into the directory `helpercode`


##
# Variable to enable/disable the batch option in Object Detection module
# Batching system allows to reconstruct trajectories from the object detection module by adding Re-Identification / Appareance matching.
# For example, if an object is not seen during some time, it can be re-ID to a previous ID if the matching score is high enough
# Use with caution if image retention is activated (See batch_system_handler.py) :
#   --> Images will only appear if an object is detected since the batching system is based on OD detection.
USE_BATCHING = False

# if __name__ == "__main__":
if True:
    print("Running object detection ... Press 'Esc' to quit")
    zed = sl.Camera()
    
    # Create a InitParameters object and set configuration parameters
    init_params = sl.InitParameters()
    init_params.coordinate_units = sl.UNIT.METER
    init_params.coordinate_system = sl.COORDINATE_SYSTEM.RIGHT_HANDED_Y_UP  
    init_params.depth_mode = sl.DEPTH_MODE.ULTRA
    init_params.depth_maximum_distance = 20
    is_playback = False                             # Defines if an SVO is used
        
    # If applicable, use the SVO given as parameter
    # Otherwise use ZED live stream
    if len(sys.argv) == 2:
        filepath = sys.argv[1]
        print("Using SVO file: {0}".format(filepath))
        init_params.svo_real_time_mode = True
        init_params.set_from_svo_file(filepath)
        is_playback = True

    status = zed.open(init_params)
    if status != sl.ERROR_CODE.SUCCESS:
        print(repr(status))
        exit()


    # Enable positional tracking module
    positional_tracking_parameters = sl.PositionalTrackingParameters()
    # If the camera is static in space, enabling this setting below provides better depth quality and faster computation
    positional_tracking_parameters.set_as_static = True
    zed.enable_positional_tracking(positional_tracking_parameters)

    # Enable object detection module
    batch_parameters = sl.BatchParameters()
    if USE_BATCHING:
        batch_parameters.enable = True
        batch_parameters.latency = 2.0
        batch_handler = BatchSystemHandler(batch_parameters.latency*2)
    else:
        batch_parameters.enable = False
    obj_param = sl.ObjectDetectionParameters(batch_trajectories_parameters=batch_parameters)
        
    obj_param.detection_model = sl.OBJECT_DETECTION_MODEL.MULTI_CLASS_BOX_FAST
    # Defines if the object detection will track objects across images flow.
    obj_param.enable_tracking = True
    zed.enable_object_detection(obj_param)

    camera_infos = zed.get_camera_information()
    # Create OpenGL viewer
    # viewer = gl.GLViewer()
    point_cloud_res = sl.Resolution(min(camera_infos.camera_configuration.resolution.width, 720), min(camera_infos.camera_configuration.resolution.height, 404)) 
    point_cloud_render = sl.Mat()
    # viewer.init(camera_infos.camera_model, point_cloud_res, obj_param.enable_tracking)
    
    # Configure object detection runtime parameters
    obj_runtime_param = sl.ObjectDetectionRuntimeParameters()
    detection_confidence = 60
    obj_runtime_param.detection_confidence_threshold = detection_confidence
    # To select a set of specific object classes
    obj_runtime_param.object_class_filter = [sl.OBJECT_CLASS.PERSON]
    # To set a specific threshold
    obj_runtime_param.object_class_detection_confidence_threshold = {sl.OBJECT_CLASS.PERSON: detection_confidence} 

    # Runtime parameters
    runtime_params = sl.RuntimeParameters()
    runtime_params.confidence_threshold = 50

    # Create objects that will store SDK outputs
    point_cloud = sl.Mat(point_cloud_res.width, point_cloud_res.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)
    objects = sl.Objects()
    image_left = sl.Mat()

    # Utilities for 2D display
    display_resolution = sl.Resolution(min(camera_infos.camera_configuration.resolution.width, 1280), min(camera_infos.camera_configuration.resolution.height, 720))
    image_scale = [display_resolution.width / camera_infos.camera_configuration.resolution.width
                 , display_resolution.height / camera_infos.camera_configuration.resolution.height]
    image_left_ocv = np.full((display_resolution.height, display_resolution.width, 4), [245, 239, 239,255], np.uint8)

    # Utilities for tracks view
    camera_config = zed.get_camera_information().camera_configuration
    tracks_resolution = sl.Resolution(400, display_resolution.height)
    track_view_generator = cv_viewer.TrackingViewer(tracks_resolution, camera_config.fps, init_params.depth_maximum_distance)
    track_view_generator.set_camera_calibration(camera_config.calibration_parameters)
    image_track_ocv = np.zeros((tracks_resolution.height, tracks_resolution.width, 4), np.uint8)

    # Will store the 2D image and tracklet views 
    global_image = np.full((display_resolution.height, display_resolution.width+tracks_resolution.width, 4), [245, 239, 239,255], np.uint8)

    # Camera pose
    cam_w_pose = sl.Pose()
    cam_c_pose = sl.Pose()

    quit_app = False

    # while(viewer.is_available() and (quit_app == False)):
    while((quit_app == False)):
        if zed.grab(runtime_params) == sl.ERROR_CODE.SUCCESS:
            # Retrieve objects
            returned_state = zed.retrieve_objects(objects, obj_runtime_param)
            
            if (returned_state == sl.ERROR_CODE.SUCCESS and objects.is_new):
                # Retrieve point cloud
                zed.retrieve_measure(point_cloud, sl.MEASURE.XYZRGBA,sl.MEM.CPU, point_cloud_res)
                point_cloud.copy_to(point_cloud_render)
                # Retrieve image
                zed.retrieve_image(image_left, sl.VIEW.LEFT, sl.MEM.CPU, display_resolution)
                image_render_left = image_left.get_data()
                # Get camera pose
                zed.get_position(cam_w_pose, sl.REFERENCE_FRAME.WORLD)

                update_render_view = True
                # update_3d_view = True
                update_tracking_view = True

                if USE_BATCHING:
                    zed.get_position(cam_c_pose, sl.REFERENCE_FRAME.CAMERA)
                    objects_batch = []
                    zed.get_objects_batch(objects_batch)
                    batch_handler.push(cam_c_pose,cam_w_pose,image_left,point_cloud,objects_batch)
                    cam_c_pose, cam_w_pose, image_left, point_cloud_render, objects = batch_handler.pop(cam_c_pose,cam_w_pose,image_left,point_cloud,objects)
                    
                    image_render_left = image_left.get_data()
                    
                    update_tracking_view = objects.is_new

                    if WITH_IMAGE_RETENTION:
                        update_render_view = objects.is_new
                        # update_3d_view = objects.is_new
                    else:
                        update_render_view = True
                        # update_3d_view = True

                # 3D rendering
                # if update_3d_view:
                    # viewer.updateData(point_cloud_render, objects)

                # 2D rendering
                if update_render_view:
                    np.copyto(image_left_ocv,image_render_left)
                    cv_viewer.render_2D(image_left_ocv,image_scale,objects, obj_param.enable_tracking)
                    global_image = cv2.hconcat([image_left_ocv,image_track_ocv])

                # Tracking view
                if update_tracking_view:
                    track_view_generator.generate_view(objects, cam_w_pose, image_track_ocv, objects.is_tracked)
                    
            cv2.imshow("ZED | 2D View and Birds View",global_image)
            cv2.waitKey(10)

        if (is_playback and (zed.get_svo_position() == zed.get_svo_number_of_frames()-1)):
            print("End of SVO")
            quit_app = True


    cv2.destroyAllWindows()
    # viewer.exit()
    image_left.free(sl.MEM.CPU)
    point_cloud.free(sl.MEM.CPU)
    point_cloud_render.free(sl.MEM.CPU)

    if USE_BATCHING:
        batch_handler.clear()

    # Disable modules and close camera
    zed.disable_object_detection()
    zed.disable_positional_tracking()

    zed.close()

## Detecting the Car

### About RoboFlow and Yolo

[Roboflow](https://roboflow.com/) is a platform which houses a vast variety of models, mainly for computer vision. It is often used because it is easy to train the models, and prediction happens over the cloud. For computer vision, Roboflow uses [YOLOv8](https://blog.roboflow.com/whats-new-in-yolov8/) (You Only Look Once), the fastest and most accurate architecture.

Roboflow has a very intuitive process to upload training images and annotate them. For models that detect (rather than just classify), the model not only must be given the image, but also the coordinates on the image of where the object in question is, and what the object is called. Roboflow allows you to create the training data fairly easily, and can then be downloaded to be trained on device.

In Version 1, the training data and the models was stored on RoboFlow, meaning anytime the code required a prediction from the model, it would make a call through the internet. This is ideal for most cases, (i.e. if the user is using a Windows laptop from 1997), however the RSS is meant to be run on an Edge computer, meaning it should have the capabilities to run the model on device. In this version, the model is trained and run on device, making it much faster.

To do this, the dataset created in RoboFlow is downloaded onto the device, and a YoloV8 model is trained with this dataset.

When the dataset is downloaded, it is stored in a folder (for example, `RC-Vehicle-Detection_v2-1`), which has some stuff
 - `test/`
    - `images/`
        - folder of the unedited images that were uploaded
    - `labels/`
        - txt files with the coords of where the object is and its classification
 - `train/`
    - same subfolders as `test`
 - `valid/`
    - same subfolders as `test`
 - `data.yaml`
    - the main datafile, has info about where the images are, what the clssifications are, etc. This file is the only file inputted when training the model 
 - `README`s

The model is then trained with this data, and saved.

### Training the Models

In [ ]:
# To Download the Dataset from Ben's RoboFlow project

# !pip install roboflow
# ^^ Don't run this line if it's already installed

rf = Roboflow(api_key=BENS_RF_APIKEY)
project = rf.workspace(BENS_RF_WORKSPACE).project(BENS_RF_PROJECT)
dataset = project.version(1).download("yolov8")


In [ ]:
## Training a custom detector

# based on this tutorial: https://learnopencv.com/train-yolov8-on-custom-dataset/

# Load the model.
model = YOLO('yolov8m.pt')
 
# # Training.
results = model.train(
   data=f'{dataset.location}/data.yaml',
   imgsz=1280,
   epochs=100,
   batch=8,
   name='yolov8m_rc_v2'
)

# For nano model, epochs 100, imgsize 640, it took 769 mins to train


In [ ]:
model = YOLO('yolov8s.pt')
 
# # Training.
results = model.train(
   data=f'{dataset.location}/data.yaml',
   # data = 'RC-Vehicle-Detection_v2-1/data.yaml',
   imgsz=640,
   epochs=100,
   batch=8,
   name='yolov8s_rc_v3'
)

# If a model ever gets interrupted in training, you can continue it by doing

# model = YOLO('runs/detect/yolov8m_rc_v2/weights/last.pt')
# model.train(resume=True)

In [ ]:
# Resuming interrupted model
# Weekend of Aug 3 it took 6800 mins
# Aug 7 night      it took 1232 mins
# Aug 10 weekend   it took 1712 mins

model = YOLO('runs/detect/yolov8m_rc_v2/weights/last.pt')

model.train(resume=True)

### Model Results


The default YOLOv8 model is the nano model, which is said to be the smallest and is therefore the fastest. However, there are 2 other models that we can use: YOLOv8s (small) and YOLOv8m (medium). These models are bigger, so in theory should be more accurate but might take longer.

Here is how each model performed:


Training
| Model Size | Epochs | Image Size | Training Time | Code Name |
| --- | --- | --- | --- | --- |
| Nano | 100 | 640 | 769 mins (12.8 hrs) | Nano 100x640 |
| Medium | 71 | 1280 | 6800 mins (4.7 days) | Med 71x1280 |
| Medium | 100 | 1280 | 9744 mins (6.8 days) | Med 100x1280 |

Testing
| Model Size | Image Source | Time per Image | Accuracy |
| --- | --- | --- | --- |
| Nano 100x640 | Photo from Ben's phone   | 0.1s | ~85% |
| Nano 100x640 | Video from my phone   | 0.3s | ~85% |
| Nano 100x640 | Frame from ZED Camera | 0.5s | ~80% |
| Med 100x1280 | Photo from Ben's phone | 0.7s | 87% |

### Testing the Models

#### Loading the new model

In [ ]:
# model = YOLO('runs/detect/yolov8n_rc_v13/weights/best.pt')
model = YOLO('runs/detect/yolov8s_rc_v3/weights/best.pt')
# metrics = model.val()
# metrics.box.map

#### Trying it on the test images

In [ ]:
results = model(glob.glob('RC-Vehicle-Detection_v2-1/test/images/*.jpg'),
                save=True,
                save_txt=True,
                save_conf=True,
                save_crop=True,
                )
# Nano: Takes 3.6 seconds to predict on 35 images, so about 0.1s per image.
# Small: Takes 9.5 seconds to predict on 35 images, so about 0.27s per image
# Medium: 50s to predict 35 images, 1.4s per image



#### Making my own score function for the model

In [ ]:

def score(realdata_path, fakedata_path):
    
    realdata_files = glob.glob(realdata_path)
    fakedata_files = glob.glob(fakedata_path)
    if (len(realdata_files) != len(fakedata_files)):
        raise "WrongFilePathException"
    confs = []
    bounds_diffs = []
    for i in range(len(realdata_files)):
        with open(realdata_files[i], 'r') as rfile:
            robj, rxl, rxr, ryl, ryr = map(float, rfile.readline().split(" "))
        with open(fakedata_files[i], 'r') as ffile:
            fobj, fxl, fxr, fyl, fyr, fconf = map(float, ffile.readline().split(" "))
            
        confs.append(fconf)
        bounds_diffs.append(abs(rxl-fxl)*100)
        bounds_diffs.append(abs(rxr-fxr)*100)
        bounds_diffs.append(abs(ryl-fyl)*100)
        bounds_diffs.append(abs(ryr-fyr)*100)
        
    print("Average Confidence:             ", avg(confs))
    print("Average Bounding Box Difference:", avg(bounds_diffs), "% of the image")

score(
    realdata_path='RC-Vehicle-Detection_v2-1/test/labels/*.txt',
    fakedata_path = 'runs/detect/predict14/labels/BEN_*.txt'
)

#### Trying the model on a video

In [ ]:
model("Videos/drive_05/drive_05_vikram2023.mp4",
      save=True,
      save_txt=True,
      save_conf=True,
      save_crop=True
)

# 15.5s video, with save,save_txt,save_conf,save_crop:
# 
# 160 seconds to predict
# 1/3 seconds to predict per frame

#### Using CV2 to detect RCC with on device model

In [ ]:
def cv2_livefeed_rfdevice(foldername = 'runs/detect/predict10/'):
  
    # define a video capture object
    vid = cv2.VideoCapture(-1) # use the camera
    vid.set(cv2.CAP_PROP_FRAME_WIDTH, 3840) 
    vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 2160)
    # 4k dimensions
    
    
    filename = foldername + 'image0.jpg'
    # Where it saves the frame
    
    latencies = [] # For understanding later

    while(True):

        # Capture the video frame by frame
        
        for _ in range(4): 
            vid.grab()
        # for some reason cv2 takes the next 5 frames and buffers them
        # this means that whats displayed is 5 frames behind
        # but when each frame takes a second to process, that's a 5 second delay
        # this makes sure its only looking at the current frame
        
        
        ret, frame = vid.read() # getting the frame
        startbig = time.time() # to calculate the latency
        
        # Zed cameras have two cameras so only use one of them
        frame = np.hsplit(frame, 2)[0]
        
        
        # do vehicle detection
        model(
            frame, 
            save=True, 
            verbose=False
        )
        # The model can save the image, but does so in some random place
        
        # display frame
        boxedframe = cv2.imread(filename)
        endbig = time.time()
        cv2.imshow('frame', boxedframe)
        
        # print(f"{round(endbig-startbig, 2)} seconds overall")
        latencies.append(endbig-startbig)
        
        
        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        
        save_frames(
            image=frame,
            dir = foldername + 'video_frames/frame_%s.jpg'
        )
        # To save the frames if you want to make a video

    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()
    
    print("Average Latency: ", round(avg(latencies), 3), "seconds")
    
    # Turn the frames into a video
    export_video(
        input_val=foldername + 'video_frames/frame_*.jpg',
        fps=4,
        output_val=foldername + 'in_the_sun.mp4'
    )
    
cv2_livefeed_rfdevice(
    foldername = 'runs/detect/predict10/' 
)
# ~0.52 seconds per loop
# ~0.42 seconds of latency

#### Using ZED camera tracking code with custom detection to recognize the vehicle

In [ ]:
# This code uses ZED Camera's code
# The benefit of this is that it uses the 2 cameras to create a 3D map
# This means it can detect the distance of the object, and it can track it across the space
# However its a bit hard to modify this code

#!/usr/bin/env python3

import sys
import numpy as np

import argparse
import torch
import cv2
import pyzed.sl as sl
from ultralytics import YOLO

from threading import Lock, Thread
from time import sleep
import time

import glob
from roboflow import Roboflow
from ultralytics import YOLO
import time
import cv2
import numpy as np

avg = lambda x: sum(x)/len(x)

# import helpercode.custom.ogl_viewer.viewer as gl
import helpercode.cv_viewer.tracking_viewer as cv_viewer
# from helpercode.batch_system_handler import *

lock = Lock()
run_signal = False
exit_signal = False


def xywh2abcd(xywh, im_shape):
    output = np.zeros((4, 2))

    # Center / Width / Height -> BBox corners coordinates
    x_min = (xywh[0] - 0.5*xywh[2]) #* im_shape[1]
    x_max = (xywh[0] + 0.5*xywh[2]) #* im_shape[1]
    y_min = (xywh[1] - 0.5*xywh[3]) #* im_shape[0]
    y_max = (xywh[1] + 0.5*xywh[3]) #* im_shape[0]

    # A ------ B
    # | Object |
    # D ------ C

    output[0][0] = x_min
    output[0][1] = y_min

    output[1][0] = x_max
    output[1][1] = y_min

    output[2][0] = x_min
    output[2][1] = y_max

    output[3][0] = x_max
    output[3][1] = y_max
    return output

def detections_to_custom_box(detections, im0):
    output = []
    for i, det in enumerate(detections):
        xywh = det.xywh[0]

        # Creating ingestable objects for the ZED SDK
        obj = sl.CustomBoxObjectData()
        obj.bounding_box_2d = xywh2abcd(xywh, im0.shape)
        obj.label = det.cls
        obj.probability = det.conf
        obj.is_grounded = False
        output.append(obj)
    return output


def torch_thread(weights, img_size, conf_thres=0.2, iou_thres=0.45):
    global image_net, exit_signal, run_signal, detections

    print("Intializing Network...")

    model = YOLO(weights)

    while not exit_signal:
        if run_signal:
            lock.acquire()

            img = cv2.cvtColor(image_net, cv2.COLOR_BGRA2RGB)
            # https://docs.ultralytics.com/modes/predict/#video-suffixes
            det = model.predict(img, save=False, imgsz=img_size, conf=conf_thres, iou=iou_thres)[0].cpu().numpy().boxes

            # ZED CustomBox format (with inverse letterboxing tf applied)
            detections = detections_to_custom_box(det, image_net)
            lock.release()
            run_signal = False
        sleep(0.01)


def main():
    global image_net, exit_signal, run_signal, detections
    weights = "runs/detect/yolov8n_rc_v13/weights/best.pt"
    img_size = 416
    conf_thres=0.4
    svo = None
    capture_thread = Thread(target=torch_thread, kwargs={
        'weights': weights, 
        'img_size': img_size, 
        "conf_thres": conf_thres
    })
    capture_thread.start()
    
    latencies = []

    print("Initializing Camera...")

    zed = sl.Camera()

    input_type = sl.InputType()
    if svo is not None:
        input_type.set_from_svo_file(svo)

    # Create a InitParameters object and set configuration parameters
    init_params = sl.InitParameters(input_t=input_type, svo_real_time_mode=True)
    init_params.coordinate_units = sl.UNIT.METER
    init_params.depth_mode = sl.DEPTH_MODE.ULTRA  # QUALITY
    init_params.coordinate_system = sl.COORDINATE_SYSTEM.RIGHT_HANDED_Y_UP
    init_params.depth_maximum_distance = 50

    runtime_params = sl.RuntimeParameters()
    status = zed.open(init_params)

    if status != sl.ERROR_CODE.SUCCESS:
        print(repr(status))
        exit()

    image_left_tmp = sl.Mat()

    print("Initialized Camera")

    positional_tracking_parameters = sl.PositionalTrackingParameters()
    # If the camera is static, uncomment the following line to have better performances and boxes sticked to the ground.
    # positional_tracking_parameters.set_as_static = True
    zed.enable_positional_tracking(positional_tracking_parameters)

    obj_param = sl.ObjectDetectionParameters()
    obj_param.detection_model = sl.OBJECT_DETECTION_MODEL.CUSTOM_BOX_OBJECTS
    obj_param.enable_tracking = True
    zed.enable_object_detection(obj_param)

    objects = sl.Objects()
    obj_runtime_param = sl.ObjectDetectionRuntimeParameters()

    # Display
    camera_infos = zed.get_camera_information()
    camera_res = camera_infos.camera_configuration.resolution
    # Create OpenGL viewer
    # viewer = gl.GLViewer()
    point_cloud_res = sl.Resolution(min(camera_res.width, 720), min(camera_res.height, 404))
    point_cloud_render = sl.Mat()
    # viewer.init(camera_infos.camera_model, point_cloud_res, obj_param.enable_tracking)
    point_cloud = sl.Mat(point_cloud_res.width, point_cloud_res.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)
    image_left = sl.Mat()
    # Utilities for 2D display
    display_resolution = sl.Resolution(min(camera_res.width, 1280), min(camera_res.height, 720))
    image_scale = [display_resolution.width / camera_res.width, display_resolution.height / camera_res.height]
    image_left_ocv = np.full((display_resolution.height, display_resolution.width, 4), [245, 239, 239, 255], np.uint8)

    # Utilities for tracks view
    camera_config = camera_infos.camera_configuration
    tracks_resolution = sl.Resolution(400, display_resolution.height)
    track_view_generator = cv_viewer.TrackingViewer(tracks_resolution, camera_config.fps, init_params.depth_maximum_distance)
    track_view_generator.set_camera_calibration(camera_config.calibration_parameters)
    image_track_ocv = np.zeros((tracks_resolution.height, tracks_resolution.width, 4), np.uint8)
    # Camera pose
    cam_w_pose = sl.Pose()

    # while viewer.is_available() and not exit_signal:
    while not exit_signal:
        if zed.grab(runtime_params) == sl.ERROR_CODE.SUCCESS:
            # -- Get the image
            
            lock.acquire()
            zed.retrieve_image(image_left_tmp, sl.VIEW.LEFT)
            
            image_net = image_left_tmp.get_data()
            lock.release()
            run_signal = True
            startbig = time.time()

            # -- Detection running on the other thread
            while run_signal:
                sleep(0.001)

            # Wait for detections
            lock.acquire()
            # -- Ingest detections
            zed.ingest_custom_box_objects(detections)
            lock.release()
            zed.retrieve_objects(objects, obj_runtime_param)

            # -- Display
            # Retrieve display data
            zed.retrieve_measure(point_cloud, sl.MEASURE.XYZRGBA, sl.MEM.CPU, point_cloud_res)
            point_cloud.copy_to(point_cloud_render)
            zed.retrieve_image(image_left, sl.VIEW.LEFT, sl.MEM.CPU, display_resolution)
            zed.get_position(cam_w_pose, sl.REFERENCE_FRAME.WORLD)

            # 3D rendering
            # viewer.updateData(point_cloud_render, objects)
            # 2D rendering
            np.copyto(image_left_ocv, image_left.get_data())
            cv_viewer.render_2D(image_left_ocv, image_scale, objects, obj_param.enable_tracking)
            global_image = cv2.hconcat([image_left_ocv, image_track_ocv])
            # Tracking view
            track_view_generator.generate_view(objects, cam_w_pose, image_track_ocv, objects.is_tracked)
            endbig = time.time()
            cv2.imshow("ZED | 2D View and Birds View", global_image)
            key = cv2.waitKey(10)
            if key == 27:
                exit_signal = True
                
            
            print(round(endbig-startbig, 2), "seconds")
            latencies.append(endbig-startbig)
        else:
            exit_signal = True

    # viewer.exit()
    exit_signal = True
    zed.close()
    
    print(round(avg(latencies), 3), "seconds per loop")




with torch.no_grad():
    main()
    
# 0.605 seconds per loop
# 0.605 seconds of latency


# in hallway, max distance = 4.3 meters
# 0.68 seconds per loop

# in outside, 5.6 max partly cloudy
# 80% outside partly cloudy


#### Report after Jul 31 2023

I created the YOLOv8 model and have it predicting on device- the performance is pretty good. When I run it on a simple image, it takes about 0.1 seconds to predict. When I run it on a video, it takes about 0.3 seconds per frame. When I run it on live feed with CV2, it takes about 0.43 seconds of processing. However, when I run the custom detector on the ZED Camera code (which processes both cameras, and tracks the object), it takes about 0.65 seconds to process. Is this latency a good amount, or is it too much?

I also tried it outside. When the car is coming towards the camera, it has about an 80% accuracy, but can identify the car pretty well, in any scenario. The car can go about 5.6 meters away before the camera stops detecting it. The algorithm was pretty good about recognizing the car in the sunlight, however when I tested it today it was about 5pm and partly cloudy so it might not be the best example- I'll check again tomorrow in more direct sunlight

#### Report After Aug 2 2023

I got a video yesterday of how the object detection performs in sunny weather when the model is on device using YoloV8 nano model, and it isn't the worst, however it isn't the best. I found that it works best  when the lighting is consistent (everything in the frame is either sunny or shady); when there is a variety, (half shady half sunny) the camera can't decide whether to make the sunny parts too sunny or the shady parts too shady, so it does some combination of both, which doesn't make the frame that good (since these kinds of webcams don't have High Dynamic Range which most of our phone cameras do). [The video (with the confidence levels) is here](https://photos.app.goo.gl/2ZW4CrC4H61wA8sZ6)

I think that this accuracy can be fixed simply by making the training data more diverse (since it was largely  just the car in a hallway, a dorm room, and outside but not that sunny), and making sure that the camera taking the photos is also diverse, or is at least the zed camera

Something that I thought about while I was on my run this morning is that right now, the code runs like this:
while true:
   read frame from zed cam
   do image recognition on the frame [~0.5 seconds to do]
   display the frame

This means that the frames are being read about once every half second. The way I had envisioned the blinking would be that within a span of 2 seconds, it would have blinked a considerable number of times in different durations to be able to convey a unique code. My concern with this is that since the camera is reading in each frame infrequently, it wouldn't register most of the blinks.

I thought of a couple of fixes for this:
 - We could have each blink last at least half a second, to ensure the edge computer can read each time the display turns off and on. The only issue with this is that the message might take several seconds to convey, which a car wouldn't have if it is moving on the road
 - We could also try parallel processing, so that the edge computer would read each frame in separately from the processing, meaning the camera will capture every time the car headlights blink. The problem with this is that the computer has to process each of those frames, which would take about 0.5s per frame, so by the time the identity is confirmed the car is long gone
 - This could also be a problem with the model itself. I noticed that the default model Roboflow comes with has very little latency, so it isn't impossible to reach very fast prediction times, however I assume that they achieved this with a large, diverse, body of training data, and possibly fine tuning the hyperparameters of the model. It is possible to get the training data, but would take a lot of manual work. 

I have been trying to train the models with different parameters, but I've only been able to train it over night. I'll do a performance review of each of the models to see which one is the fastest and most accurate.

I was also thinking, since I now can edit the code on Ben's display, we could try to display a QR code which links to a url hosted on a server (that could submit a POST request), which the edge computer is connected to. I figured this might be good since there should be some optimized models out there to scan QR codes, so it should help with latency, however this still requires the board

## Detecting the headlights

### Background For Detecting the Headlights

In [this paper from Innopolis University in Russia](https://github.com/TechToker/Vehicle-rear-lights-analyser/blob/master/FinalReport.pdf), researchers intended detect if the car in front of them were braking or not. 

The researchers use YOLOv4 to detect whether a car is present for each frame, and then cropp the frame to just the car.

To detect the presense of brakelights, they convert each image into YCrCb space, then removes all pixels that aren't pure red. They utilize the fact that the taillights are symmetrical in cars to weed away noise from other cars. They then apply a bounding box over the taillights.

To determine if the car is braking, it determines the brightness in the bounding boxes. It's important to note that it has to see what the car looks like when braking vs not braking to know which is brighter, and thus be able to distinguish when it's braking.

Our algorithm can utilize soemthing similar, but will have to identify the headlights a different way since headlights aren't red. Potentially, a ML model could be used to determine the headlights' position. Determining when its on vs off would be the same as the Innopolis paper.



[This article](https://ieeexplore.ieee.org/abstract/document/6761567) talks about how the researchers detected headlights to track vehicles. They first converted the image to a binary image (meaning grayscaled, then converted everything above a certain threshold to 255 and below that threshold to 0), and detected the blobs. The then detected the blobs of white areas, and considered each blob a candidate for one of the pair of headlights. Finally, they used the property that headlights are symmetrical to pair 2 of the blobs and determine if they are symmetrical (using some equations).
From this, they applied it to counting and tracking vehicles in night time / low light. It remains to be seen whether this technique would work in the daytime (since converting the image to binary might not work as well / produce more blobs in higher light), but a method like this could work on the prototype car (since it has the red display board).

### Method 1: Using a dataset from RoboFlow

#### Report From Aug 1 2023 (kind of just a flood of random thoughts)

Steps that need to be done now.

I need to find a good way to detect headlights.
There is a headlights ML algorithm that I can train it on
That needs to be done over night

I also would need to make the headlights work on Ben's display
I need Ben's arduino code to make that happen

I also need to find a way to decode the headlights
I need the display to make that work?

Overnight I should train the algorithms
Then tmr I should test to see if it works in the day
Then find the intensity values

Also take a picture of actual car headluights, play around with those in different formats (Binary, YCrCb, etc), also needs to be done at home

#### Download the dataset

In [ ]:
# !pip install roboflow

# Downloading a headlights detection dataset

from roboflow import Roboflow
rf = Roboflow(api_key="2Vs9PCO5LGDCSkI0huRq")
project = rf.workspace("nours-g8unb").project("vehicles_night")
dataset = project.version(3).download("yolov8")


#### Train the model

In [ ]:
## Training a custom detector

# based on this tutorial: https://learnopencv.com/train-yolov8-on-custom-dataset/


 
# Load the model.
model = YOLO('yolov8n.pt')
 
# # Training.
results = model.train(
   data=f'{dataset.location}/data.yaml',
   imgsz=1280,
   epochs=100,
   batch=8,
   name='yolov8n_headlights_v1'
)

# Took 1258 mins for imgsz=1280, epochs 100, nano model

#### Reopen the model

In [ ]:
model = YOLO('runs/detect/yolov8n_headlights_v1/weights/best.pt')

#### Test the model

In [ ]:
results = model(glob.glob('Videos/headlight_test/*.jpg'),
                save=True,
                # save_txt=True,
                save_conf=True,
                # save_crop=True,
                )
# Takes 5.9 seconds to predict on 6 images, so about 1s per image.

# Accuracy really bad
# Scrap this method

### Method 2: Don't detect headlights, detect flashes of light

#### Report Aug 7 2023

To Detect headlights, one approach is to detect the vehicle like normal, and simply detect when the vehicle's lights flash by detecting when the vehicle region becomes bright. This way, you don't have to recognize where a vehicle's headlights are, you can instead act like a human and only notice when the lights flash.

A couple ways to detect a blinking light are the following
 - Take the average brightness of the cropped image, compare them frame by frame, and see when the biggest different is, and that is when it blinks ([explained here](https://stackoverflow.com/questions/38030170/best-way-to-detect-a-light-blinking-from-video-footage-via-python))
    - You could do this on the normal cropped image, a greyscaled version, or a thresholded version
    - This has problems in that it might be influenced by other factors, like shade, or the vehicle moving
    - For my testing, it does work, however I set the threshold to 200. When the headlights blinked, the mean image value was 11, and otherwise it was 0.5
 - Take the difference of 2 consecutive full frames ([explained here](https://stackoverflow.com/questions/8877228/opencv-detect-blinking-lights)) to detect when the light blinks
    - This should only be done with thresholded images (otherwise it causes seizures)
    - This won't work because the vehicle moves so it will capture the difference between the vehicle's last positions, not the headlights, [as shown here](https://photos.app.goo.gl/CMKZ16492xYHcSuj6)
 - Take the difference of 2 consecutive _cropped_ frames to detect when the light blinks
    - This won't work because the cropped frames have very slightly different dimensions, and it would be hard to align them as it would get the vehicle change, not the headlights

I did the first option because it's the only one that can actually work. I did this by detecting the vehicle, cropping the vehicle to just that, converting the cropped image to only black or white (I call it thresholding the image, [here is a video where I try a bunch of different thresholds](https://photos.app.goo.gl/YyxE9YHTYNSLU2sCA)), and finding the average light value of the cropped image.

#### Open and run the RC detection model

In [ ]:
model = YOLO('runs/detect/yolov8n_rc_v13/weights/best.pt')

In [ ]:
results = model(['Videos/PXL_20230807_143124752.MP.jpg', 'Videos/PXL_20230807_143126993.MP.jpg'],
                save=True,
                # save_txt=True,
                save_conf=True,
                save_crop=True,
                )

#### Taking frames, detecting the RC, Thresholding them to detect blinking

In [ ]:
blinkinglights = sorted(glob.glob(
    # "runs/detect/predict10/crops/RC-Car/*.jpg"
    "runs/detect/predict5/video_frames/*.jpg"
))

for framefileind in range(1, len(blinkinglights)):
    framefile = blinkinglights[framefileind]
    lastframefile = blinkinglights[framefileind-1]
    threshold = 100
    # frame = cv2.imread(framefile)
    # frame = cv2.cvtColor(cv2.imread(framefile), cv2.COLOR_BGR2GRAY)
    _, frame = cv2.threshold(cv2.cvtColor(cv2.imread(framefile), cv2.COLOR_BGR2GRAY), threshold, 255, cv2.THRESH_BINARY)
    # lastframe = cv2.imread(lastframefile)
    # lastframe = cv2.cvtColor(cv2.imread(lastframefile), cv2.COLOR_BGR2GRAY)
    _, lastframe = cv2.threshold(cv2.cvtColor(cv2.imread(lastframefile), cv2.COLOR_BGR2GRAY), threshold, 255, cv2.THRESH_BINARY)
    framediff = frame-lastframe
    
    cv2.imwrite('Videos/aug72023vids/diff_frames%04d.jpg' % framefileind, framediff)
    cv2.imshow('diff frame', framediff)
    # cv2.imwrite('Videos/aug72023vids/cropped_vehicle_blinking_normal%04d.jpg' % framefileind, frame)
    # cv2.imshow('frame', frame)
    print(f' {framefileind}/{len(blinkinglights)}, mean: {frame.mean()}')
    # frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # threshold = 70

    # # Apply threshold to the masked grayscale image
    # _, thresh = cv2.threshold(frame_gray, threshold, 255, cv2.THRESH_BINARY)

    # cv2.imshow('frame', thresh)
    cv2.waitKey(0)
    
export_video(
    input_val='Videos/aug72023vids/diff_frames*.jpg',
    fps=16,
    output_val='Videos/aug72023vids/frame_differences.mp4'
)
    
# for threshold in range(1, 256):
#     framefile = blinkinglights[0]
#     # lastframefile = blinkinglights[framefileind-1]
    
#     # frame = cv2.imread(framefile)
#     # frame = cv2.cvtColor(cv2.imread(framefile), cv2.COLOR_BGR2GRAY)
#     _, frame = cv2.threshold(cv2.cvtColor(cv2.imread(framefile), cv2.COLOR_BGR2GRAY), threshold, 255, cv2.THRESH_BINARY)
#     # lastframe = cv2.imread(lastframefile)
#     # lastframe = cv2.cvtColor(cv2.imread(lastframefile), cv2.COLOR_BGR2GRAY)
#     # _, lastframe = cv2.threshold(cv2.cvtColor(cv2.imread(lastframefile), cv2.COLOR_BGR2GRAY), threshold, 255, cv2.THRESH_BINARY)
#     # framediff = frame-lastframe
    
#     # cv2.imshow('diff frame', framediff)
#     frame = cv2.putText(frame, "Threshold: %s" % threshold, (5,15), cv2.FONT_HERSHEY_PLAIN, 0.75, (100,100,100), 1)
#     cv2.imshow('frame', frame)
#     cv2.imwrite('Videos/aug72023vids/threshold%03d.jpg' % threshold, frame)
#     # print(f' {framefileind}/{len(blinkinglights)}')
#     # frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     # threshold = 70

#     # # Apply threshold to the masked grayscale image
#     # _, thresh = cv2.threshold(frame_gray, threshold, 255, cv2.THRESH_BINARY)

#     # cv2.imshow('frame', thresh)
#     cv2.waitKey(0)

# export_video(
#     input_val='Videos/aug72023vids/threshold*.jpg',
#     fps=20,
#     output_val='Videos/aug72023vids/different_thresholds.mp4'
# )

cv2.destroyAllWindows()

#### Functions to help

In [ ]:
def crop_image(ogimage, cropboxes):
    if len(cropboxes) == 0: np.array([])
    cropbox = cropboxes[0] 
    tly, tlx, bry, brx = map(int, list(cropbox))
    newimg = ogimage[tlx:brx, tly:bry]
    return newimg    
    
def threshold_image(img, threshold=200):
    _, newimg = cv2.threshold(
        cv2.cvtColor(
            img, 
            cv2.COLOR_BGR2GRAY
        ), 
        threshold, 255, cv2.THRESH_BINARY
    )
    
    return newimg

def detect_onoff_headlights_v1(img):
    # greyscale and threshold the image
    threshimg = threshold_image(
        img=img, threshold=HL_THRESHOLD
    )
    
    
    return int(threshimg.mean() > ONOFF_THRES), threshimg

#### Main function on live feed

In [ ]:
model = YOLO('runs/detect/yolov8n_rc_v13/weights/best.pt')

In [ ]:
def cv2_livefeed_rfdevice_wheadlights(model):
    
    # model = YOLO('runs/detect/yolov8n_rc_v13/weights/best.pt')
  
    # define a video capture object
    vid = cv2.VideoCapture(-1) # use the camera
    vid.set(cv2.CAP_PROP_FRAME_WIDTH, 3840) 
    vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 2160)
    # 4k dimensions
    
    foldername = 'runs/detect/predict12/'
    filename = foldername + 'image0.jpg'
    latencies = []
    count = 0
    headlight_pattern = []
    
    while(True):

        # Capture the video frame by frame
        
        for _ in range(4): 
            vid.grab()
        # for some reason cv2 takes the next 5 frames and buffers them
        # this means that whats displayed is 5 frames behind
        # but when each frame takes a second to process, that's a 5 second delay
        # this makes sure its only looking at the current frame
        
        
        ret, frame = vid.read()
        startbig = time.time()
        # Zed cameras have two cameras so only use one of them
        frame = np.hsplit(frame, 2)[0]
        
        
        # do vehicle detection
        results = model(
            frame, 
            save=True, 
            verbose=False
        )


        # crop the image to just the vehicle
        cropped_image = crop_image(
            ogimage = frame, cropboxes=results[0].boxes.xyxy.numpy()
        )
        
        hl_onoff, threshimg = detect_onoff_headlights_v1(
            img=cropped_image
        )
        headlight_pattern.append(hl_onoff)

        
        # cv2.imshow('boxed frame', cv2.imread(filename))
        # cv2.imshow('cropped frame', cropped_image)
        cv2.imshow('thresholded frame', threshimg)
        
        
        endbig = time.time()
        
        # print(f"{round(endbig-startbig, 2)} seconds overall")
        latencies.append(endbig-startbig)
        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        current_time = datetime.now().strftime("%H_%M_%S_%f")
        cv2.imwrite(foldername + 'video_frames/frame_%s.jpg' % current_time, frame)

    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()
    
    print("Average Latency: ", round(avg(latencies), 3), "seconds")
    print(headlight_pattern)

    
cv2_livefeed_rfdevice_wheadlights(model)

# Average latency: 0.392s

#### Notes about the Encryption Architecture (Aug 21 2023)



The encryption should be based on something you know about the vehicle and have a random element (like how credit card numbers have the digits in the prefix to know the company of the card, and a pseudo-random sequence of numbers in the suffix)

Marco can create a function, whose inputs are information about the vehicle, and output is a string (or list) of ones and zeros, which will be displayed on the headlights (0 for off, 1 for on, flashed at a certain frequency). 

The main constraint is that the vehicle will be moving 10mph - 30mph, meaning there will only be 10bits - 30bits of information that can be transmitted. However, the number of bits generated depends on the speed of the car, so a question becomes **how will the camera system know what the speed of the car will be, to make the most optimal encrypted string?**

The camera system and the vehicle will both need to know what the correct code is.

One approach:
 - Implement the function to allow each vehicle and infrastructure to have this random function, and both objects will have the same seed to generate the random sequence. One question is then **how will both objects have that seed?**
    - Option 1: The seed could be something public about the car, like the license plate but that's not secure
    - Option 2: Seed could be sent securely from the camera to the vehicle, so then **what's the best way to send it?**
        - Option 1: Asymmetric Encryption, but that is computationally expensive, and not quantum secure (which isn't a big problem now)
        - Option 2: Symmetric Encryption, however that requires both the sender and receiver to know a common private key, meaning all vehicles will have to be registerd in some database. **Is this feasible?**


## Confirming the headlight pattern


#### How it works

(Approach Version 2)

This algorithm takes in the detected headlight sequence, `detected`, compare it to the originally generated headlight sequence, `generated`, and see if the detected sequence can confirm the identity of the vehicle.

One example of the generated sequence is `1,0,1,1,0`.

One could think that you can directly compare the two lists, and see if they are the same. However, a problem with this is that the camera is reading in frames at twice the frame rate as the screen is flashing. This means, for the example sequence above, the sequences would be \
`1...0...1...1...0..` (generated) \
`1,1,0,0,1,1,1,1,0,0` (detected)

However it isn't as simple as doubling each element in the sequence because sometimes, the camera will read in frames faster or slower than what is predicted (it depends on rounding, loop-by-loop latencies, etc). This means the sequences might look more like this: \
`1...0.....1...1.0..` (generated) \
`1,1,0,0,0,1,1,1,0,0` (detected)

In addition, the camera might not be completely accurate in detecting the headlights, for example something might obstruct the view of the headlights (a bird might fly in front of the camera, for example): \
`1...0...1...1...0..` (generated) \
`1,1,0,0,1,0,1,1,0,0` (detected)

The algorithm needs to be able to handle small problems: the camera and screen aren't completely synced; an object might obstruct the headlights. 

My algorithm uses a [Dynamic Time Warp (DTW)](https://dtaidistance.readthedocs.io/en/latest/usage/dtw.html) to handle the second. A DTW can compare two temporal sequences, which may slightly differ. The DTW is very good at quantifying how many slight differences there are, by giving back a distance between the two sequences. For example, here are two sequences, where DTW gives where the sequences match up, and guives a distance of sqrt(2) because there are 2 places where they differ:

![image](figs/inplace_diff.png) \
Distance: sqrt(2)

Where the DTW fails is when the two sequences are on two different time scales, it doesn't try to preserve the length of each leg (a consecutive stretch of 1s or 0s) of the pattern. This image shows two time scales, which shouldn't be deemed the same in our context, but DTW gives a distance of 0:

![image](figs/timescale_diff.png) \
Distance: 0

This means in addition to the DTW, I need my own algorithm to detect how in-sync the time scales are.

My algorithm takes the connected points (called a path) that it determines, and checks the ratio of lengths of each leg between the detected sequence and the generated sequence. If they are roughly equal to the ratio between the headlight-flashing interval and the camera-frame-read interval, then it is good. For example if the length of each leg is: \
6, 4, 8, 7, 2 for the detected sequence, and \
3, 2, 4, 3, 1 for the generated sequence, that that is correct (the 7 vs 3 is considered a time-scale error, but is still deemed correct since the headlights might have meant to flash for 3 seconds and the camera read 3.5).

However if the lengths were \
6, 4, 8, 8, 2 for the detected sequence, and \
3, 2, 4, 3, 1 for the generated sequence, then that is an error because the headlights must have been flashing for 4 seconds, not 3.

This approach considers both types of errors, and accounts for them while still reporting if the sequences do not verify the vehicle.

#### Testing DTW

In [ ]:
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis
import random
s1 = [1, 0, 0,0,0,0,0,0, 1, 0, 1,1, 0, 1, 1, 1]

s2 = [1,0,1,0,1,0,1]

# s1 = [0,0,1,1,0,0,0,0,1,1,0,0,1,1,1,1,1,1]
# s2 = [0,  1,  0,  0,  1,  0,  1,  1,  1]
path = dtw.warping_path(
    s1, s2
)
print(path)
dtwvis.plot_warping(s1,s2, path, filename="warp.png")
print(dtw.distance(s1,s2))

In [ ]:
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis

s2 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
s1 = [0,1,1,1,0,0,1,0,1,0,0]

path = dtw.warping_path(
    s1, s2
)
print(path)
dtwvis.plot_warping(s1,s2, path, filename="trial1.png")
print(dtw.distance(s1,s2))

In [ ]:
def compare_sequences_v1(detected, generated):
    from dtaidistance import dtw
    from dtaidistance import dtw_visualisation as dtwvis

    path = dtw.warping_path(
        detected, generated
    )
    print(path)
    dtwvis.plot_warping(detected, generated, path, filename="veh_trial.png")
    print(dtw.distance(detected, generated))
    # the lower the better distance    

If I want to account for the timescales for both sequences being skewed, I need to write my own algorithm

#### My thought process for making the algorithm (Aug 23 2023)




DTW is good at quantifying how far off the pattern is (solely in terms of when 0 changes to 1 and vice versa) but not how many 0s and 1s there are in a row. To solve this, I can create my own algorithm that quantifies how far off the pattern is (in terms of numbers in a row).


There are 2 things I will need to account for:
1. FPS is off: the camera reads in images at half second intervals, however for Nyquest's theorm, the flashes of the headlights have to be at 1s intervals. This means if the headlights flash `0,1,0`, the camera will read `0,0,1,1,0,0`. There may be problems with this, like if the camera reads in faster than 1/2s, so it may read in `0,0,1,1,1,0,0`. My algorithm will quantify these discrepensies.
2. If a bird flies through the camera frame, for example, the camera feed of `0,1,1,0,0,1,0,1` may read `0,1,1,0,0,0,0,1`. My algorithm will have to count the number of times this happens.


To do this, I think I can use DTW to find the distances of the paths, which should give which point on path 1 corrosponds to a point on path 2. I can make my own function to compare how many points corrospond to the paths at each segment.

#### Function to do it

In [ ]:
def compare_sequences_v2(detected, generated, caminterval, flashinterval, skipfirst=False, save='warp.png'):
    path = dtw.warping_path(
        detected, generated
    )
    distance = dtw.distance(
        detected, generated
    )
    dtwvis.plot_warping(detected, generated, path, filename=save)
    
    
    detectedlen = {}
    generatedlen = {}
    
    
    
    lastg = -1
    for ind, dg in enumerate(path):
        d, ge = dg
        
        if (ind > 0 and generated[path[ind-1][1]] == generated[path[ind][1]]):
            g = lastg
            
        else:
            g = ge
            lastg = g
            
        if (ind > 0 and path[ind-1][1] != path[ind][1] and generated[path[ind-1][1]] == generated[path[ind][1]]):
            generatedlen[g]+=1
        else:
            generatedlen[g] = 1
            
        if (g not in detectedlen): detectedlen[g] = 0
        detectedlen[g]+=1
    
    weirdcounter = 0
    for g in detectedlen:
        if skipfirst: 
            if g == 0: continue
        weirdcounter += int(abs(detectedlen[g] - ( generatedlen[g] * (flashinterval / caminterval) )) / (flashinterval/caminterval))
        
    print("Generated:", generated)
    print("Path:", path)
    print("Dist:", distance)
    for d,g in path:
        print(d,g)    
    print("DetectedLen: ", detectedlen)
    print("GeneratedLen:", generatedlen)
    print(weirdcounter, "/", len(generated))
    
    # return weirdcounter/len(generated), distance**2 / len(generated)
    return weirdcounter, distance**2
        
camerr, birderr = compare_sequences_v2(
    detected= [1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1],
    generated=[1, 0, 1, 0, 1],
    caminterval=1,
    flashinterval=1,
    save='figs/timescale_diff.png'
)

print(camerr, birderr)

#### Algorithm report

The alg works pretty well

When the vehicle is displaying the right pattern, it shows anywhere from 0-3 distance (depending on how many birds fly through), and has a timescale error of 0.

When the vehicle is displaying the wrong pattern, it shows 5+ distance, and has a timescale error avging around 7.

# Final Code

This function uses functions from other parts of the code, so they may have to be run first before this can be run

In [ ]:
def cv2_livefeed_rfdevice_wheadlights(model):
    
    # model = YOLO('runs/detect/yolov8n_rc_v13/weights/best.pt')
  
    # define a video capture object
    vid = cv2.VideoCapture(-1) # use the camera
    vid.set(cv2.CAP_PROP_FRAME_WIDTH, 3840) 
    vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 2160)
    # 4k dimensions
    
    foldername = 'runs/detect/predict12/'
    filename = foldername + 'image0.jpg'
    latencies = []
    count = 0
    headlight_pattern = []
    
    while(True):

        # Capture the video frame by frame
        
        for _ in range(4): 
            vid.grab()
        # for some reason cv2 takes the next 5 frames and buffers them
        # this means that whats displayed is 5 frames behind
        # but when each frame takes a second to process, that's a 5 second delay
        # this makes sure its only looking at the current frame
        
        
        ret, frame = vid.read()
        startbig = time.time()
        # Zed cameras have two cameras so only use one of them
        frame = np.hsplit(frame, 2)[0]
        
        
        # do vehicle detection
        results = model(
            frame, 
            save=True, 
            verbose=False
        )


        # crop the image to just the vehicle
        cropped_image = crop_image(
            ogimage = frame, cropboxes=results[0].boxes.xyxy.numpy()
        )
        
        hl_onoff, threshimg = detect_onoff_headlights_v1(
            img=cropped_image
        )
        headlight_pattern.append(hl_onoff)

        
        # cv2.imshow('boxed frame', cv2.imread(filename))
        # cv2.imshow('cropped frame', cropped_image)
        cv2.imshow('thresholded frame', threshimg)
        
        
        endbig = time.time()
        
        # print(f"{round(endbig-startbig, 2)} seconds overall")
        latencies.append(endbig-startbig)
        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        current_time = datetime.now().strftime("%H_%M_%S_%f")
        cv2.imwrite(foldername + 'video_frames/frame_%s.jpg' % current_time, frame)

    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()
    
    print("Average Latency: ", round(avg(latencies), 3), "seconds")
    print(headlight_pattern)
    
    # compare_sequences_v1(
    #     detected=headlight_pattern,
    #     generated= [0,1,1,1,0,0,1,0,1,0,0]
    # )
    
    camerr, birderr = compare_sequences_v2(
        detected=headlight_pattern,
        # generated= [0,1,1,1,0,0,1,0,1,0,0],
        generated= [random.randint(0,1) for _ in range(10)],
        caminterval=avg(latencies),
        flashinterval=1,
        skipfirst=True
    )
    
    print(camerr, birderr)
    
cv2_livefeed_rfdevice_wheadlights(model)